<ul>
  <li>El Mundo (https://elmundo.es) </li>
    <ul> 
        <li>Salud: https://www.elmundo.es/ciencia-y-salud/salud.html </li>
        <li>Tecnología: https://www.elmundo.es/tecnologia.html  </li>
        <li>Ciencia: https://www.elmundo.es/ciencia-y-salud/ciencia.html</li>
    </ul>
  <li>El País (https://elpais.com/)</li>
    <ul> 
        <li>Sanidad: https://elpais.com/noticias/sanidad/ </li>
        <li>Tecnología: https://elpais.com/tecnologia/</li>
        <li> Ciencia: https://elpais.com/ciencia/</li>
    </ul>
  <li>20 Minutos (https://www.20minutos.es/) </li>
    <ul> 
        <li>Salud: https://www.20minutos.es/salud/ </li>
        <li>Tecnología: https://www.20minutos.es/tecnologia/ </li>
        <li>Ciencia: https://www.20minutos.es/ciencia/ </li>
    </ul>
 
</ul>

In [38]:
from bs4 import BeautifulSoup
import requests
import os

In [67]:
# El Pais

diario = "https://elpais.com"
diario_texto = "elPais"
counter_dictionary = {}

def elPaisPalabrasLlaves(soup):
    palabras_llaves = ""
    separador = ", "
    # Hacemos un lista con las palabras llave
    html_llaves = soup.find("ul", class_="_df _ls")
    # Solo extraemos los enlaces
    enlaces_llaves = html_llaves.find_all("a")
    # Sacamos el texto de los enlaces y y lo añadimos a la lista 
    for enlace in enlaces_llaves:
        palabras_llaves = palabras_llaves + enlace.text + separador
    # Quitamos el ultimo separador
    palabras_llaves = palabras_llaves[:-(len(separador))]
    return palabras_llaves

def elPaisTexto(soup):
    content = ""
    # div con todo el texto
    text = soup.find("div", class_="a_c")
    # buscar todos los p
    paragraphs = text.find_all("p")
    # Sacamos el contenido de p y lo añadimos a content
    for paragraph in paragraphs:
        content = content + paragraph.text + "\n"
    # print(content)
    return content

def elPaisTitulo(soup):
    return soup.find("h1", class_="a_t").text

def elPaisEntradilla(soup):
    return soup.find("h2", class_="a_st").text

def elPaisFecha(soup):
    date_length = 10 # 2021-12-03T18:09:39.529Z
    return soup.find("a", id="article_date_p")["data-date"][:date_length]

def guardarFichero(contenido, fecha, diario_texto, categoria):
    # calcular numero (nnn)
    counter = 1
    if fecha in counter_dictionary[categoria]:
        counter = counter_dictionary[categoria][fecha]
        counter_dictionary[categoria][fecha] += 1
    else:
        counter_dictionary[categoria][fecha] = 2
    
    # /diario/categoria/categoria.fecha.nnn.txt
    abs_path = os.getcwd()
    rel_path =  diario_texto + "/" + categoria + "/"
    nombre_fichero = categoria + "." + fecha + "." + str(counter).zfill(3) + ".txt"
    path = os.path.join(abs_path, rel_path, nombre_fichero)
    try:
        with open(path, 'x') as fichero:
            fichero.write(contenido)
        print ("File " + nombre_fichero + " created!")
    except IOError:
        print("File " + nombre_fichero + " already exists...")


def elPaisWebScraping(pagina, diario_texto, categoria):
    soup = BeautifulSoup(pagina, 'html.parser')

    palabras_llaves = elPaisPalabrasLlaves(soup)
    titulo = elPaisTitulo(soup)
    entradilla = elPaisEntradilla(soup)
    texto = elPaisTexto(soup)
    fecha = elPaisFecha(soup)

    content = palabras_llaves + "\n\n" + titulo + "\n\n" + entradilla + "\n\n" + texto
    guardarFichero(content,fecha,diario_texto, categoria)
    return content

def elPais(diario, diario_texto, categoria, url):
    counter_dictionary[categoria] = {}
    pagina = requests.get(url)
    soup = BeautifulSoup(pagina.content, 'html.parser')

    # sacar todas los enlaces de los articulos de la pagina categorial
    enlaces = []
    for article in soup.find_all("article"):
        enlaces.append(diario + article.find("a")['href'])

    # processar cada articulo
    for enlace in enlaces: 
        articulo = requests.get(enlace)
        try:
            elPaisWebScraping(articulo.content, diario_texto, categoria)
        except:
            print("Error procesando articulo, saltando el articulo " + enlace)



# processar las categorias: salud, ciencia y tecnologia
categoria = "salud"
url = diario + "/noticias/sanidad/"
elPais(diario, diario_texto, categoria, url)

categoria = "ciencia"
url = diario + "/ciencia/"
elPais(diario, diario_texto, categoria, url)

categoria = "tecnologia"
url = diario + "/tecnologia/"
elPais(diario, diario_texto, categoria, url)


File tecnologia.2021-12-06.001.txt created!
File tecnologia.2021-12-07.001.txt created!
File tecnologia.2021-12-07.002.txt created!
File tecnologia.2021-12-07.003.txt created!
File tecnologia.2021-12-06.002.txt created!
File tecnologia.2021-12-04.001.txt created!
File tecnologia.2021-12-05.001.txt created!
File tecnologia.2021-12-05.002.txt created!
File tecnologia.2021-12-04.002.txt created!
File tecnologia.2021-12-04.003.txt created!
File tecnologia.2021-12-05.003.txt created!
File tecnologia.2021-12-03.001.txt created!
File tecnologia.2021-12-03.002.txt created!
File tecnologia.2021-12-03.003.txt created!
File tecnologia.2021-12-03.004.txt created!
File tecnologia.2021-12-01.001.txt created!
File tecnologia.2021-11-29.001.txt created!
File tecnologia.2021-11-26.001.txt created!
File tecnologia.2021-11-24.001.txt created!
File tecnologia.2021-10-25.001.txt created!
Error procesando articulo, saltando el articulo https://elpais.com/tecnologia/con-proposito/2021-12-01/asi-es-la-luz-del